In [1]:
# You only need to run this once per machine
! pip install -q -U bitsandbytes
! pip install -q -U git+https://github.com/huggingface/transformers.git
! pip install -q -U git+https://github.com/huggingface/peft.git
! pip install -q -U git+https://github.com/huggingface/accelerate.git
! pip install -q -U datasets scipy ipywidgets matplotlib

In [ ]:
# You only need to run this once per machine
! pip install -q trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# I was getting error about the GPU driver that need to be updated, so I downgrated the torch version
! pip uninstall torch
! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [1]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [2]:
import numpy as np
import pandas as pd

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

import utilities

In [3]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

Detected kernel version 4.14.287, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [4]:
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

Fine-tune

In [7]:
# load the tables
conversation_df = pd.read_csv("annotated_trees_101.csv")
conversation_df = conversation_df.rename({'Unnamed: 0': "index"}, axis=1)
df_only_bad_tone = pd.read_csv("bad_tone_nodes_with_generated_messages_chat_gpt_3_5_turbo.csv")
df_only_bad_tone["neg branch path"] = df_only_bad_tone["neg branch path"].apply(eval) # converts the list string into list

In [8]:
# Generate train dataset
train_data = list()
for i in range(df_only_bad_tone.shape[0]):
    index = df_only_bad_tone["index"].iloc[i]
    prompt = utilities.generate_branch_for_negative_tone_prompt_for_mistral(df_only_bad_tone, conversation_df, node_index=i)
    label = df_only_bad_tone["generated_moderation"].iloc[i]
    train_data.append((index, prompt, label))

dataset_df = pd.DataFrame(train_data, columns=["index", 'prompt', 'label'])
dataset_df.to_pickle("train_dataset.pkl")

In [9]:
from datasets import load_dataset
mod_dataset = load_dataset("pandas", data_files="train_dataset.pkl", split='train')
mod_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['index', 'prompt', 'label'],
    num_rows: 1732
})

In [10]:
def generate_mod_prompt(data_point):
    return f"""{data_point['prompt']}\n{data_point['label']}</s>"""


text_column = [generate_mod_prompt(ex) for ex in mod_dataset]
mod_dataset = mod_dataset.add_column("full_prompt", text_column)

mod_dataset = mod_dataset.shuffle(seed=1234)  # Shuffle dataset here
mod_dataset = mod_dataset.map(lambda samples: tokenizer(samples["full_prompt"]), batched=True)
mod_dataset = mod_dataset.train_test_split(test_size=0.2, seed=1234)
train_data = mod_dataset["train"]
test_data = mod_dataset["test"]

Map:   0%|          | 0/1732 [00:00<?, ? examples/s]

In [11]:
mod_dataset

DatasetDict({
    train: Dataset({
        features: ['index', 'prompt', 'label', 'full_prompt', 'input_ids', 'attention_mask'],
        num_rows: 1385
    })
    test: Dataset({
        features: ['index', 'prompt', 'label', 'full_prompt', 'input_ids', 'attention_mask'],
        num_rows: 347
    })
})

In [12]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [13]:
import bitsandbytes as bnb

def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
  lora_module_names = set()
  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

modules = find_all_linear_names(model)
print(modules)

['v_proj', 'gate_proj', 'down_proj', 'up_proj', 'q_proj', 'o_proj', 'k_proj']


In [14]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [15]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 20971520 | total: 7262703616 | Percentage: 0.2888%


In [21]:
#new code using SFTTrainer
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="full_prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=1,
        max_steps=100,
        learning_rate=2.5e-5,
        logging_steps=1,
        save_steps=10,
        save_strategy="steps",
        eval_steps=10,
        evaluation_strategy="steps",
        do_eval=True,
        output_dir="outputs_mine",
        optim="paged_adamw_8bit",
        bf16=True,
        
        logging_dir="./logs_mine"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Detected kernel version 4.14.287, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss,Validation Loss
10,1.799100,1.808764
20,1.546500,1.755077
30,1.702700,1.713680
40,1.547100,1.672189
50,1.696000,1.630933
60,1.460600,1.593421
70,1.529400,1.561854
80,1.563400,1.539072
90,1.548600,1.524203
100,1.588400,1.518224


/home/ec2-user/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ec2-user/.local/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ec2-user/.local/lib/python3.9/site-packages/torch/utils/checkpoi

TrainOutput(global_step=100, training_loss=1.6634824967384338, metrics={'train_runtime': 2940.5416, 'train_samples_per_second': 0.272, 'train_steps_per_second': 0.034, 'total_flos': 3.4846549875572736e+16, 'train_loss': 1.6634824967384338, 'epoch': 0.58})

Load fined tuned model

In [23]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [24]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "outputs_mine/checkpoint-100")

In [14]:
def get_completion(prompt, model, tokenizer):
    model_inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda")

    generated_ids = model.generate(**model_inputs, max_new_tokens=300, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    decoded = tokenizer.batch_decode(generated_ids)
    
    return decoded[0]

def get_mod_message_generation_function(model, tokenizer):
    def generate_moderation_message(row):
        print('sample:', row.name)
        answer = get_completion(row['prompt'], model=model, tokenizer=tokenizer)
        splits = answer.split('[/INST]')
        if len(splits) != 2:
            return 'model produced illegal output..'
        return splits[-1].rstrip('</s>')
    
    return generate_moderation_message

In [12]:
train_df, test_df = train_data.to_pandas(), test_data.to_pandas()

In [28]:
fine_tuned_model_moderation = get_mod_message_generation_function(ft_model, eval_tokenizer)
test_df['fine_tuned_model_moderation'] = test_df.apply(fine_tuned_model_moderation, axis=1)

sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7


sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 91
sample: 92
sample: 93
sample: 94
sample: 95
sample: 96
sample: 97
sample: 98
s

Base model

In [14]:
base_model_id = "mistralai/Mistral-7B-Instruct-v0.2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
base_model_generation = get_mod_message_generation_function(model, tokenizer)
test_df['base_model_moderation'] = test_df.apply(base_model_generation, axis=1)

sample: 0
sample: 1
sample: 2
sample: 3
sample: 4
sample: 5
sample: 6
sample: 7
sample: 8
sample: 9
sample: 10
sample: 11
sample: 12
sample: 13
sample: 14
sample: 15
sample: 16
sample: 17
sample: 18
sample: 19
sample: 20
sample: 21
sample: 22
sample: 23
sample: 24
sample: 25
sample: 26
sample: 27
sample: 28
sample: 29
sample: 30
sample: 31
sample: 32
sample: 33
sample: 34
sample: 35
sample: 36
sample: 37
sample: 38
sample: 39
sample: 40
sample: 41
sample: 42
sample: 43
sample: 44
sample: 45
sample: 46
sample: 47
sample: 48
sample: 49
sample: 50
sample: 51
sample: 52
sample: 53
sample: 54
sample: 55
sample: 56
sample: 57
sample: 58
sample: 59
sample: 60
sample: 61
sample: 62
sample: 63
sample: 64
sample: 65
sample: 66
sample: 67
sample: 68
sample: 69
sample: 70
sample: 71
sample: 72
sample: 73
sample: 74
sample: 75
sample: 76
sample: 77
sample: 78
sample: 79
sample: 80
sample: 81
sample: 82
sample: 83
sample: 84
sample: 85
sample: 86
sample: 87
sample: 88
sample: 89
sample: 90
sample: 9

In [29]:
test_df = test_df[["index", "prompt", "label", "base_model_moderation", "fine_tuned_model_moderation"]]
test_df.to_csv("test_with_fine_tuned_moderations_mine.csv", index=False)